In [394]:
#gs25 세븐 cu 이마트24
#유동인구 데이터 통신사
#올빼미 데이터 m버스는 kt데이터로 유동인구
#trade off
# http://gs25.gsretail.com/gscvs/ko/store-services/locations
# 컨투롤u 컨트롤f해서 csrf찾아보기 매번다르고 접속할때 매번 다르게받는

In [3]:
import requests
import zipfile #zipfile c말고도 java로 된 파이썬 json
import os
import pandas as pd
import shutil 
import pickle
from bs4 import BeautifulSoup as BS
gs25_url = "http://gs25.gsretail.com/gscvs/ko/store-services/locationList"
r=requests.get(gs25_url)
#crsf보안코드 #보이는거므로 get으로한

In [396]:
bs=BS(r.text)

In [4]:
import requests 
from bs4 import BeautifulSoup as BS 

r= requests.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")

bs = BS(r.text)

#<form id="CSRFForm" action="/gscvs/ko/store-services/locations" method="post"><input type="hidden" name="CSRFToken" value="406690e8-0340-4931-b372-4b5519577c3c" />
bs.find("input", {'name': "CSRFToken"})


<input name="CSRFToken" type="hidden" value="6ef15760-62fb-426f-9a2c-c36c33be0071"/>

In [5]:
from bs4 import BeautifulSoup as BS 
from tqdm import tqdm
r= requests.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")
payload = {"pageNum": "1",
"pageSize": "100", #몇개뽑을건지 테스트용 5 전체는 100
"searchShopName": "",
"searchSido": "",
"searchGugun": "",
"searchDong": "",
"searchType": "",
"searchTypeService": "0",
"searchTypeToto": "0",
"searchTypeCafe25": "0",
"searchTypeInstant": "0",
"searchTypeDrug": "0",
"searchTypeSelf25": "0",
"searchTypePost": "0",
"searchTypeATM": "0",
"searchTypeWithdrawal": "0",
"searchTypeTaxrefund": "0",
"searchTypeSmartAtm": "0",
"searchTypeSelfCookingUtensils": "0",
"searchTypeDeliveryService": "0",
"searchTypeParcelService": "0",
"searchTypePotatoes": "0",
"searchTypeCardiacDefi": "0",
"searchTypeFishShapedBun": "0",}

bs = BS(r.text)
total=[]
#<form id="CSRFForm" action="/gscvs/ko/store-services/locations" method="post"><input type="hidden" name="CSRFToken" value="406690e8-0340-4931-b372-4b5519577c3c" />
with requests.Session() as s: #같은 토큰으로만 받기위해서
    r = s.get("http://gs25.gsretail.com/gscvs/ko/store-services/locations")
    bs = BS(r.text)
    token=bs.find("input", {'name': "CSRFToken"})['value']
    target=f"http://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={token}"
    # print(bs.find("input", {'name': "CSRFToken"}))
    for x in tqdm(range(1,175)): #테스트용 3 전체 175
        # if x% 10 ==0 : print(x)
        payload["pageNum"]=x
        r2=s.post(target,data=payload)
        total.append(pd.DataFrame(eval(r2.json())['results']))
        
   
    
    

    


100%|█████████████████████████████████████████████████████████████████████████████████| 174/174 [00:20<00:00,  8.50it/s]


In [ ]:
r2.text

In [ ]:
# gs25.shopCode.unique().size

In [ ]:
gs25=pd.concat(total)

In [ ]:
with open("./gs25.pkl","wb") as f:
     pickle.dump(gs25,f)


In [ ]:
with open("./gs25.pkl","rb") as f:
    gs25 = pickle.load(f)

In [ ]:
gs25.shopCode.unique().size

In [ ]:
gs25.reset_index(drop=True,inplace=True)

In [ ]:
gs25

In [1]:
#세븐일레븐
input_dosi=["서울"]
input_gogoun=["강남구","강서구"]
eleven_url="https://www.7-eleven.co.kr/util/storeLayerPop.asp"
eleven_payload={"storeLaySido": "서울",
"storeLayGu": "중구",
"hiddentext": "none"}
eleven_r = requests.post(eleven_url, data=eleven_payload)
total_eleven_name=[]
total_eleven_add=[]
eleven_bs=BS(eleven_r.text)
eleven_bs

NameError: name 'requests' is not defined

In [113]:

# new=[]
# eleven_bs
gogun_url="https://www.7-eleven.co.kr/library/asp/StoreGetGugun.asp"
all_dosi=["강원도","경기도","경상남도","경상북도","광주","대구","대전","부산","서울","세종","울산","인천","전라남도","전라북도","제주도","충청남도","충청북도"]
all_gogun=[]
gogun_payload={"Sido": "서울",
"Gu": "강남구",
"selName": "storeLayGu"}
# test=requests.post(gogun,data=payload)
# bs1=BS(test.text)
# bs1
seven_name_list=[]
seven_add_list=[]
seven_service_list=[]
for i in tqdm(range(len(all_dosi))):
    gogun_payload["Sido"]=all_dosi[i]
    gogun_payload["Gu"]=""
    test=requests.post(gogun_url,data=gogun_payload)
    bs1=BS(test.text)
    for x in bs1.findAll("option"):
        if x.text!="구/군":
            # print(all_dosi[i]+" "+x.text,end=" , ")
            # x.test에 ~~시가 들어간
            # all_gogun.append(x.text)
            eleven_url="https://www.7-eleven.co.kr/util/storeLayerPop.asp"
            eleven_payload={"storeLaySido": "서울",
            "storeLayGu": "중구",
            "hiddentext": "none"}
            eleven_payload["storeLaySido"]=all_dosi[i]
            eleven_payload["storeLayGu"]=x.text
            # print(all_dosi[i]+"  "+ x.text)
            test1=requests.post(eleven_url,data=eleven_payload)
            bs2=BS(test1.text)
            # print(bs1)
            # break
            for y in bs2.find("div", class_ = "list_stroe").findAll("li"):
                # print(x.find("a").text)
                # print(x)
                # print(x.findAll("span").text)
                # print(x)
                # print(all_dosi[i],x.text,len(y.findAll("img")))
                just_service_temp=[]
                for z in range(len(y.findAll("img"))):
                    just_service_temp.append(y.findAll("img")[z]['src'].split("/")[-1].split("_")[-1].replace(".png", ""))
                    
                    # print(y.findAll("img")[z]['src'].split("/")[-1].split("_")[-1].replace(".png", ""),end=" , ")
                seven_service_list.append(just_service_temp)
                # print(y.findAll("img")[0]['src'].split("/")[-1].split("_")[-1].replace(".png", ""),end=" , ")
                try:
                    temp=y.find("a").text.split("\n")
                    # print(y.find("a").text.strip().split("\n")[0])
                    # print(y.find("a").text.strip().split("\n")[3].strip())
                    just_list_temp=[]
                    for k in range(0,len(temp)-2):
                        if len(temp[k])>3:
                            just_list_temp.append(temp[k])
                    seven_name_list.append(just_list_temp[0].strip())
                    seven_add_list.append(just_list_temp[-1].strip())
                except:
                    pass
               
                
                
                # print(x)
                # print(x.find("a").text.strip().split("\n")[0].strip()+" "+x.find("a").text.strip().split("\n")[3].strip())
                # print(x.find("a").text.strip().split("\n")[3].strip())
                # print(seven_service_list)
                # break
                
                # seven_name_list.append(x.find("a").text.strip().split("\n")[0].strip())
                # seven_add_list.append(x.find("a").text.strip().split("\n")[-1].strip())
            #     for x in tqdm(eleven_bs.find("div", class_ = "list_stroe").findAll("li")):
            # # print(x.find("a").text)
            #     a=x.find("a").text
            #     total_eleven_name.append(x.find("a").text.strip().split("\n")[0].strip())
            #     total_eleven_add.append(x.find("a").text.strip().split("\n")[-1].strip())
        # for i in tqdm(range(len(all_dosi))):
        #     for j in range(len(all_gogun)):
        #         payload["storeLaySido"]=all_dosi[i]
        #         payload["storeLayGu"]=all_gogun[j]
        #         test=requests.post(eleven_url,data=eleven_payload)
        #         bs1=BS(test.text)
        #         print(bs1)
        #         break
        # for x in bs1.find("div", class_ = "list_stroe").findAll("li"):
        # # print(x.find("a").text)
        #     a=x.find("a").text
        #     total_eleven_name.append(a.strip()..split("\n")[0].strip())
        #     total_eleven_add.append(a.strip().split("\n")[-1].strip())
            
      
                    
        
# for x in tqdm(bs1.findAll("option")):
#     print(x.text)

100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [01:29<00:00,  5.24s/it]


In [112]:
just_list_temp

['충주힐스테이트점   ', '충청북도 충주시 호암수청1로 29\xa0(호암동, LH힐스테이트)']

In [123]:
# print(all_dosi)
# len(seven_service_list)  #12874
# len(seven_name_list)
# len(seven_add_list)
# seven_service_list
# seven_service_list=seven_service_list[:len(seven_service_list)-2]
len(seven_service_list)
# df=pd.DataFrame({"name": seven_name_list, "add" : seven_add_list, "service" : seven_service_list})
    
    # index=["name","add","service"], data=[seven_name_list,seven_add_list,seven_service_list])
# pd.DataFrame({'temp':temp, 'temp1':temp}


12860

In [117]:
df.head(10)

,0,1,2,3,4,5,6,7,8,9,...,12858,12859,12860,12861,12862,12863,12864,12865,12866,12867
name,강릉5주공점,강릉강문스타점,강릉강문점,강릉강변로점,강릉경포그린점,강릉경포로점,강릉경포오션점,강릉곶감점,강릉관솔점,강릉교동센터점,...,충주충원대로점,충주충청대로점,충주탄금대로점,충주터미널점,충주푸르지오점,충주호암점,충주호암LH점,충주힐스테이트점,None,None
add,강원도 강릉시 월대산로 43,강원도 강릉시 창해로350번길 21 (강문동),"강원도 강릉시 해안로 298-7 B블록 101호, 102호(강문동)",강원도 강릉시 강변로 262,"강원도 강릉시 경포로463번길 37, 제2호",강원특별자치도 강릉시 경포로463번길 23 1층안현동,강원도 강릉시 경포로463번길 30-5. 1층 108호,강원도 강릉시 곶감전길 9 (성남동),강원도 강릉시 관솔길 79-1,강원특별자치도 강릉시 율곡초교길 21 (교동),...,충청북도 충주시 충원대로 164 1층,충청북도 충주시 주덕읍 충청대로 2323,충청북도 충주시 탄금대로 224 (칠금동),충청북도 충주시 봉계1길 49,"충청북도 충주시 봉계8길 2 , 1층",충청북도 충주시 호암중앙2로 32 101호(호암동),충청북도 충주시 호암수청2로 56,"충청북도 충주시 호암수청1로 29 (호암동, LH힐스테이트)",None,None
service,"[medical, parcel, fedex, cd]","[parcel, fedex]","[24h, cafe, medical, parcel, fedex, smartpick]","[parcel, fedex]","[parcel, fedex, softcon]","[parcel, fedex]","[parcel, fedex]","[24h, cafe, chicken, medical, toto, reserve, p...","[parcel, fedex, softcon]","[parcel, fedex]",...,"[24h, cafe, medical, reserve, parcel, atm, fed...","[parcel, fedex]","[parcel, fedex]","[parcel, fedex]","[parcel, fedex]","[parcel, fedex]","[parcel, fedex]","[24h, cafe, chicken, medical, reserve, parcel,...","[parcel, fedex]","[parcel, fedex]"


In [ ]:
# #세븐일레븐
# input_dosi=["서울"]
# input_gogoun=["강남구","강서구"]
# eleven_url="https://www.7-eleven.co.kr/util/storeLayerPop.asp"
# eleven_payload={"storeLaySido": "서울",
# "storeLayGu": "중구",
# "hiddentext": "none"}
# eleven_r = requests.post(eleven_url, data=eleven_payload)
# total_eleven_name=[]
# total_eleven_add=[]
# eleven_bs=BS(eleven_r.text)
# eleven_bs

In [ ]:
all_gogun

In [402]:
eleven_url="https://www.7-eleven.co.kr/util/storeLayerPop.asp"
eleven_payload={"storeLaySido": "서울",
"storeLayGu": "중구",
"hiddentext": "none"}
for i   in tqdm(range(len(all_dosi))):
    for j in range(len(all_gogun)):
        payload["storeLaySido"]=all_dosi[i]
        payload["storeLayGu"]=all_gogun[j]
        test=requests.post(eleven_url,data=eleven_payload)
        bs1=BS(test.text)
        print(bs1)
        break
        # for x in bs1.find("div", class_ = "list_stroe").findAll("li"):
        # # print(x.find("a").text)
        #     a=x.find("a").text
        #     total_eleven_name.append(a.strip().split("\n")[0].strip())
        #     total_eleven_add.append(a.strip().split("\n")[-1].strip())
            
   

  6%|████▉                                                                               | 1/17 [00:00<00:05,  3.03it/s]

<html><body><div class="layer_pop store_find" id="layer" style="width:960px">
<div class="layer_header">
<h1 class="tit_store">지점(STORE) 찾기</h1>
</div><!-- //layer_header -->
<div class="layer_body">
<div class="skip_gMap">
<a href="#skip_googleMap">지도영역 건너뛰기</a>
</div>
<div class="wrap_map">
<iframe border="2" id="mapifrmAction" marginheight="0" marginwidth="0" name="mapifrmAction" scrolling="yes" src="/util/mapifrmAction.asp" style="width:638px;height:586px;border:0" title="다음 API빈프레임"></iframe>
</div>
<div class="wrap_store" id="skip_googleMap">
<form action="" id="storeForm" method="post" name="storeForm" onkeypress="return event.keyCode != 13">
<input id="storeLocationGu" name="storeLocationGu" title="기본설정 구/군" type="hidden" value="중구"/>
<input id="hiddentext" name="hiddentext" type="hidden" value="none"/>
<input id="storeLayOption" name="storeLayOption" type="hidden" value=""/>
<ul class="tab_store">
<li class="on"><a href="#">지역별 지점</a></li>
<li><a href="#">지점명 검색</a></li>
</ul>

 12%|█████████▉                                                                          | 2/17 [00:00<00:04,  3.17it/s]

<html><body><div class="layer_pop store_find" id="layer" style="width:960px">
<div class="layer_header">
<h1 class="tit_store">지점(STORE) 찾기</h1>
</div><!-- //layer_header -->
<div class="layer_body">
<div class="skip_gMap">
<a href="#skip_googleMap">지도영역 건너뛰기</a>
</div>
<div class="wrap_map">
<iframe border="2" id="mapifrmAction" marginheight="0" marginwidth="0" name="mapifrmAction" scrolling="yes" src="/util/mapifrmAction.asp" style="width:638px;height:586px;border:0" title="다음 API빈프레임"></iframe>
</div>
<div class="wrap_store" id="skip_googleMap">
<form action="" id="storeForm" method="post" name="storeForm" onkeypress="return event.keyCode != 13">
<input id="storeLocationGu" name="storeLocationGu" title="기본설정 구/군" type="hidden" value="중구"/>
<input id="hiddentext" name="hiddentext" type="hidden" value="none"/>
<input id="storeLayOption" name="storeLayOption" type="hidden" value=""/>
<ul class="tab_store">
<li class="on"><a href="#">지역별 지점</a></li>
<li><a href="#">지점명 검색</a></li>
</ul>

 18%|██████████████▊                                                                     | 3/17 [00:00<00:04,  3.15it/s]

...

<html><body><div class="layer_pop store_find" id="layer" style="width:960px">
<div class="layer_header">
<h1 class="tit_store">지점(STORE) 찾기</h1>
</div><!-- //layer_header -->
<div class="layer_body">
<div class="skip_gMap">
<a href="#skip_googleMap">지도영역 건너뛰기</a>
</div>
<div class="wrap_map">
<iframe border="2" id="mapifrmAction" marginheight="0" marginwidth="0" name="mapifrmAction" scrolling="yes" src="/util/mapifrmAction.asp" style="width:638px;height:586px;border:0" title="다음 API빈프레임"></iframe>
</div>
<div class="wrap_store" id="skip_googleMap">
<form action="" id="storeForm" method="post" name="storeForm" onkeypress="return event.keyCode != 13">
<input id="storeLocationGu" name="storeLocationGu" title="기본설정 구/군" type="hidden" value="중구"/>
<input id="hiddentext" name="hiddentext" type="hidden" value="none"/>
<input id="storeLayOption" name="storeLayOption" type="hidden" value=""/>
<ul class="tab_store">
<li class="on"><a href="#">지역별 지점</a></li>
<li><a href="#">지점명 검색</a></li>
</ul>

 94%|██████████████████████████████████████████████████████████████████████████████     | 16/17 [00:05<00:00,  2.85it/s]

<html><body><div class="layer_pop store_find" id="layer" style="width:960px">
<div class="layer_header">
<h1 class="tit_store">지점(STORE) 찾기</h1>
</div><!-- //layer_header -->
<div class="layer_body">
<div class="skip_gMap">
<a href="#skip_googleMap">지도영역 건너뛰기</a>
</div>
<div class="wrap_map">
<iframe border="2" id="mapifrmAction" marginheight="0" marginwidth="0" name="mapifrmAction" scrolling="yes" src="/util/mapifrmAction.asp" style="width:638px;height:586px;border:0" title="다음 API빈프레임"></iframe>
</div>
<div class="wrap_store" id="skip_googleMap">
<form action="" id="storeForm" method="post" name="storeForm" onkeypress="return event.keyCode != 13">
<input id="storeLocationGu" name="storeLocationGu" title="기본설정 구/군" type="hidden" value="중구"/>
<input id="hiddentext" name="hiddentext" type="hidden" value="none"/>
<input id="storeLayOption" name="storeLayOption" type="hidden" value=""/>
<ul class="tab_store">
<li class="on"><a href="#">지역별 지점</a></li>
<li><a href="#">지점명 검색</a></li>
</ul>

100%|███████████████████████████████████████████████████████████████████████████████████| 17/17 [00:05<00:00,  3.02it/s]

<html><body><div class="layer_pop store_find" id="layer" style="width:960px">
<div class="layer_header">
<h1 class="tit_store">지점(STORE) 찾기</h1>
</div><!-- //layer_header -->
<div class="layer_body">
<div class="skip_gMap">
<a href="#skip_googleMap">지도영역 건너뛰기</a>
</div>
<div class="wrap_map">
<iframe border="2" id="mapifrmAction" marginheight="0" marginwidth="0" name="mapifrmAction" scrolling="yes" src="/util/mapifrmAction.asp" style="width:638px;height:586px;border:0" title="다음 API빈프레임"></iframe>
</div>
<div class="wrap_store" id="skip_googleMap">
<form action="" id="storeForm" method="post" name="storeForm" onkeypress="return event.keyCode != 13">
<input id="storeLocationGu" name="storeLocationGu" title="기본설정 구/군" type="hidden" value="중구"/>
<input id="hiddentext" name="hiddentext" type="hidden" value="none"/>
<input id="storeLayOption" name="storeLayOption" type="hidden" value=""/>
<ul class="tab_store">
<li class="on"><a href="#">지역별 지점</a></li>
<li><a href="#">지점명 검색</a></li>
</ul>

In [401]:
len(total_eleven_name)

670503

In [400]:
len(total_eleven_add)

670503

In [ ]:

for x in tqdm(eleven_bs.find("div", class_ = "list_stroe").findAll("li")):
    # print(x.find("a").text)
    a=x.find("a").text
    total_eleven_name.append(a.strip().split("\n")[0].strip())
    total_eleven_add.append(a.strip().split("\n")[-1].strip())
    # type(a)
    # total_eleven.
    # title = x.find("a", class_="title ellipsis").text.strip()
    # break

In [ ]:
# for x in tqdm(eleven_bs.find("div", class_ = "list_stroe").findAll("li")):
#     print(x)
#     break

# 포함점
# ,
# ,
# ,
# ,
# ,
# 자세한주소

In [ ]:
# input_dosi=[{"서울": ["강남구","강서구"]}]
# # input_gogoun=["강남구","강서구"]
# for i in input_dosi:
#     print(i)

In [ ]:


for x in tqdm(eleven_bs.find("div", class_ = "list_stroe").findAll("li")):
    # print(x.find("a").text)
    a=x.find("a").text
    total_eleven_name.append(a.strip().split("\n")[0].strip())
    total_eleven_add.append(a.strip().split("\n")[-1].strip())
    # type(a)
    # total_eleven.
    # title = x.find("a", class_="title ellipsis").text.strip()
    # break

In [ ]:
total_eleven_name

In [399]:
total_eleven_add

['서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구 마른내로 146\xa01층1호',
 '서울특별시 중구

In [ ]:
eleven_df=pd.DataFrame({"이름": total_eleven_name, "add":total_eleven_add})
eleven_df

In [ ]:
print(a)

In [ ]:
a.strip()

In [ ]:
# eleven_bs = BS(eleven_r.text)
# # r2.text
# # r2.json()
# # type(r2.json())
# # eval(r2.json())
# # eval(r2.json())['results']
# # import pickle

In [ ]:
# eleven_bs
eleven_r.text
# total.append(pd.DataFrame(eval(r2.json())['results']))

In [ ]:
import json
# eval(eleven_r.text.json())
json.loads(eleven_r.text)
# type(eleven_r.text)